In [0]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from pylab import random


**20 NewsGroups Dataset**

In [0]:
news = fetch_20newsgroups(subset='train', shuffle=True)
number_of_documents = 20       # taking few documents to cluster

observation_matrix = CountVectorizer(stop_words='english').fit_transform(news.data[:number_of_documents])
M = observation_matrix


labels = news.target
K = np.unique(labels).shape[0]

m = M.shape[0]    # No. of documents
n = M.shape[1]    # No. of words(excluding english language stop words)

In [64]:
print(K)

20


In [65]:
print(m)

20


In [66]:
print(n)

1866


In [0]:
def EStep():
    for i in range(0, m):
        for j in range(0, n):
            denominator = 0;
            for k in range(0, K):
                p[i, j, k] = theta[k, j] * lamda[i, k];
                denominator += p[i, j, k];
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0;
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator;
                    

In [0]:
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, m):
        for j in range(0, n):
            tmp = 0
            for k in range(0, K):
                tmp += theta[k, j] * lamda[i, k]
            if tmp > 0:
                loglikelihood += M[i, j] * np.log(tmp)
    return loglikelihood


In [0]:
def initializeParameters():
    for i in range(0, m):
        normalization = sum(lamda[i, :])
        for j in range(0, K):
            lamda[i, j] /= normalization;

    for i in range(0, K):
        normalization = sum(theta[i, :])
        for j in range(0, n):
            theta[i, j] /= normalization;

In [0]:
# lamda[i, j] : p(tj|di)
lamda = random([m, K])

# theta[i, j] : p(wj|ti)
theta = random([K, n])

# p[i, j, k] : p(tk|di,wj)
p = np.zeros([m, n, K])
initializeParameters()

In [0]:
# EM algorithm
maxIteration = 3


In [0]:
for i in range(0, maxIteration):
    EStep()
    #Mstep
    # update theta # theta[i, j] = p(wj|ti)
    for k in range(0, K):
        denominator = 0
        for j in range(0, n):
            theta[k, j] = 0
            for i in range(0, m):
                theta[k, j] += M[i, j] * p[i, j, k]
            denominator += theta[k, j]
        if denominator == 0:
            for j in range(0, n):
                theta[k, j] = 1.0 / n
        else:
            for j in range(0, n):
                theta[k, j] /= denominator
        
    # update lamda # lamda[i, j] = p(tj|di)

    t_mat = np.zeros((m, K))
    for i in range(0, m):
      for k in range(0, K):
          lamda[i, k] = 0
          denominator = 0
          lamda[i, -1] = 0

          for j in range(0, n):
            lamda[i, k] += M[i, j] * p[i, j, k]
            denominator += M[i, j];
              
          if denominator == 0:
            lamda[i, k] = 1.0 / K 
          else:
            lamda[i, k] /= denominator  

          t_mat[i][k] = lamda[i, k]



In [73]:
print(t_mat)

[[4.64346592e-04 2.02955207e-01 1.58022348e-02 2.64706596e-04
  1.41425646e-02 1.35673902e-01 2.95768886e-02 6.86579737e-03
  1.06839708e-02 6.24805199e-03 1.08002252e-01 2.61750551e-02
  2.55422611e-03 1.04786938e-01 1.69108733e-01 3.98837840e-02
  9.48343720e-02 2.70795373e-02 4.73586274e-03 1.61569637e-04]
 [1.94435471e-03 1.48567971e-02 2.96796963e-03 3.86133905e-01
  4.10058417e-03 7.59795133e-02 1.76645188e-03 4.16954583e-02
  1.66794988e-03 1.93517493e-03 3.05214451e-02 6.88723243e-05
  2.67296796e-02 1.21110091e-01 2.63386748e-02 1.21035046e-03
  3.46759801e-04 4.65200286e-02 7.74009744e-02 1.36704965e-01]
 [2.02006704e-02 2.77352891e-02 5.58897678e-04 8.54273707e-04
  6.73353602e-03 5.79766566e-02 4.24870339e-03 4.87760943e-03
  7.56671479e-02 6.70210783e-02 1.69221420e-01 5.48611676e-02
  7.47934159e-02 1.44353469e-03 9.53306009e-02 1.86632672e-01
  3.66111258e-03 7.34398917e-02 7.37656486e-02 9.76673263e-04]
 [1.20384827e-01 1.21550900e-01 5.01058880e-02 1.82916766e-02
  2.9

**Now we will assign the LATENT TOPICS for each document**

In [0]:
t_star = []
for i in range(0, m):
  t_star.append(np.argmax(t_mat[i]))


In [75]:
print(t_star)

[1, 3, 15, 8, 14, 12, 5, 9, 8, 4, 2, 16, 4, 17, 10, 4, 16, 6, 10, 4]


In [76]:
print(np.size(t_star))

20


In [101]:
#Cluster numbers and Document numbers are starting from 0
for k in range(0, K):
  doc = []
  for i in range(0, m):
    if t_star[i] == k:
      doc.append(i)
  if doc == []:
    print("Documents in cluster ",k,"     : NO DOCUMENTS FOUND")
  else:
    print("Documents in cluster ",k," are : ","Document Number ",doc)

Documents in cluster  0      : NO DOCUMENTS FOUND
Documents in cluster  1  are :  Document Number  [0]
Documents in cluster  2  are :  Document Number  [10]
Documents in cluster  3  are :  Document Number  [1]
Documents in cluster  4  are :  Document Number  [9, 12, 15, 19]
Documents in cluster  5  are :  Document Number  [6]
Documents in cluster  6  are :  Document Number  [17]
Documents in cluster  7      : NO DOCUMENTS FOUND
Documents in cluster  8  are :  Document Number  [3, 8]
Documents in cluster  9  are :  Document Number  [7]
Documents in cluster  10  are :  Document Number  [14, 18]
Documents in cluster  11      : NO DOCUMENTS FOUND
Documents in cluster  12  are :  Document Number  [5]
Documents in cluster  13      : NO DOCUMENTS FOUND
Documents in cluster  14  are :  Document Number  [4]
Documents in cluster  15  are :  Document Number  [2]
Documents in cluster  16  are :  Document Number  [11, 16]
Documents in cluster  17  are :  Document Number  [13]
Documents in cluster  